<a href="https://colab.research.google.com/github/sagar9926/Natural-Language-Processing/blob/main/TFIDF_in_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Credits : https://towardsdatascience.com/a-gentle-introduction-to-calculating-the-tf-idf-values-9e391f8a13e5

## What is TF IDF?

Intuitively, to understand what text is about, we look for words that occur frequently. Term frequency covers that aspect by capturing the number of times each word occurs in the text. However, in this article, there are 200+ occurrences of the word ‘the’ and only 55 occurrences of the word TF (which includes the code used as well). To downgrade the relative importance of words that occur all too frequently, an inverse weighting is introduced to scale down the words that occur too frequently. This inverse weighting is referred to as Inverse Document Frequency. Together, TF-IDF captures the relative importance of words in a set of documents or a collection of texts.

In an applied business context, the text classification problem is one of the common problems in NLP. In text classification problems, the algorithms have to predict the topic based on a predefined set of topics it has trained on. In 2018, Google released a text classification framework based on 450K experiments on a few different text sets. Based on the 450K experiments, Google found that when the number of samples/number of words < 1500, TF IDF was the best way to represent text. When you have a smallish sample size for a relatively common problem, it helps to try out TF IDF.



## Overview

We will be using a beautiful poem by the mystic poet and scholar Rumi as our example corpus. First, we will calculate TF IDF values for the poem using TF IDF Vectorizer from the sklearn package. Then, we will pull apart the various components and work through various steps involved in calculating TF-IDF values. Mathematical calculations and Python code will be provided for each step.
So, let’s go!

In [1]:
corpus =  ["you were born with potential",
"you were born with goodness and trust",
"you were born with ideals and dreams",
"you were born with greatness",
"you were born with wings",
"you are not meant for crawling, so don't",
"you have wings",
"learn to use them and fly"
]

## Looking ahead to the final output

We will be decimating the beautiful poem into mysterious decimals in this step. But, hey, after all, we are trying to demystify these decimals by understanding the calculations involved in TF-IDF. As mentioned before, it is quite easy to derive through sklearn package.

In [2]:
#transform the tf idf vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tf_idf_vect = TfidfVectorizer()
X_train_tf_idf = tf_idf_vect.fit_transform(corpus)
terms = tf_idf_vect.get_feature_names()

In the matrix below, each row represents a sentence from the above poem. Each column represents a unique word in the poem in alphabetical order. As you can see, there are lot of zeros in the matrix. So, a memory-efficient sparse matrix is used for representing this. I have converted it to a data frame for ease of visualization.

In [3]:
import pandas as pd
tfidf_matrix = pd.DataFrame(columns = ["sentence"] + terms)
tfidf_matrix["sentence"] = corpus
for i in range(len(corpus)):
  tfidf_matrix.iloc[i,1:] = X_train_tf_idf[i].toarray()[0]


tfidf_matrix

,sentence,and,are,born,crawling,don,dreams,fly,for,goodness,greatness,have,ideals,learn,meant,not,potential,so,them,to,trust,use,were,wings,with,you
0,you were born with potential,0,0,0.383289,0,0,0,0,0,0,0,0,0,0,0,0,0.682895,0,0,0,0,0,0.383289,0,0.383289,0.304834
1,you were born with goodness and trust,0.37764,0,0.293087,0,0,0,0,0,0.522185,0,0,0,0,0,0,0,0,0,0,0.522185,0,0.293087,0,0.293087,0.233096
2,you were born with ideals and dreams,0.37764,0,0.293087,0,0,0.522185,0,0,0,0,0,0.522185,0,0,0,0,0,0,0,0,0,0.293087,0,0.293087,0.233096
3,you were born with greatness,0,0,0.383289,0,0,0,0,0,0,0.682895,0,0,0,0,0,0,0,0,0,0,0,0.383289,0,0.383289,0.304834
4,you were born with wings,0,0,0.413022,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.413022,0.616716,0.413022,0.328481
5,"you are not meant for crawling, so don't",0,0.372697,0,0.372697,0.372697,0,0,0.372697,0,0,0,0,0,0.372697,0.372697,0,0.372697,0,0,0,0,0,0,0,0.166366
6,you have wings,0,0,0,0,0,0,0,0,0,0,0.725164,0,0,0,0,0,0,0,0,0,0,0,0.607744,0,0.323703
7,learn to use them and fly,0.307727,0,0,0,0,0,0.425512,0,0,0,0,0,0.425512,0,0,0,0,0.425512,0.425512,0,0.425512,0,0,0,0


Let us interpret the numbers we have received so far. As you may have noticed, the words “you were born” are repeated throughout the poem. So, we anticipate that these words will not be getting high TF-IDF scores. If you look at the values for those three words, you can see that most often they get between 0.2 and 0.3.

Let us look at Document 0— You were born with potential. The word potential stands out. If you look at the various TF-IDF values in the first row in the matrix, you will see that the word potential has the highest TF-IDF value.

Let us look at Document 4 (row 5): You were born with wings. Again, same as before, the word “wings” has the highest value in that sentence.
Notice that the word “wings” appears also in Document 6. TF-IDF value for the word “wings” in Document 6 is different to TF-IDF value for the word “wings” in Document 4. 

In Document 6, the word “wings” is deemed less important than the word “have” in Document 6.

We will be focusing on applying the calculations on the words __wings__ and __potential__ in particular, to derive the values highlighted in red in the matrix displayed above.

We will break apart the various components and then put them back together. We will do this in three steps:

* Step 1: Derive term frequency values from scratch

* Step 2: Derive inverse document frequency values from scratch

* Step 3: Aggregate the above two values using multiplication and normalization

## Step 1 : Calculate term frequency values

The term frequency is pretty straight forward. It is calculated as the number of times the words/terms appear in a document.

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(corpus)
terms = count_vect.get_feature_names()

countvec_matrix = pd.DataFrame(columns = ["sentence"] + terms)
countvec_matrix["sentence"] = corpus
for i in range(len(corpus)):
  countvec_matrix.iloc[i,1:] = X_train_counts[i].toarray()[0]


countvec_matrix

,sentence,and,are,born,crawling,don,dreams,fly,for,goodness,greatness,have,ideals,learn,meant,not,potential,so,them,to,trust,use,were,wings,with,you
0,you were born born born with potential,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,1
1,you were born with goodness and trust,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,1
2,you were born with ideals and dreams,1,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,1
3,you were born with greatness,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1
4,you were born with wings,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1
5,"you are not meant for crawling, so don't",0,1,0,1,1,0,0,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,1
6,you have wings,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1
7,learn to use them and fly,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,0,0,0


Let us find out term frequency for the words potential and wings.

* Term Frequency for the word potential in Doc 0= 1

* Term Frequency for the word wings in Doc 4= 1

* Term Frequency for the word wings in Doc 6 = 1

However, if we just take the term frequency as a measure of importance, we fall into one major pitfall as represented by the tweet below from Terrible Maps representing the most frequent word in each state in the United States.

![](https://pbs.twimg.com/media/D627MpgW4AAqTNW?format=jpg)

## Step 2: Inverse Document Frequency

We need a mechanism to tone down the relative importance of the words that occur all too frequently in all the documents. Enter the __Inverse Document Frequency__ . Intuitively, if a word appears in all documents, then it may not play such a big part in differentiating between the documents.

Similar to Term Frequency,

```
Document Frequency(term t) = number of documents with the term t/ total number of documents = d(t)/n

Inverse Document Frequency = total number of documents / number of documents with the term t = n / d(t)


If a word appears in all the documents, we want it at the bottom of the range of 0–1. So, a logarithmic scale intuitively makes sense to be used here as log 1 is 0.

 However, there are some practical considerations such as avoiding the infamous divide by 0 error, 1 is added to the denominator.

 
Inverse Document frequency for the default settings in TF IDF vectorizer in sklearn is calculated as below (default settings have smooth_idf=True that adds “1” to the numerator and denominator as if an extra document was seen containing every term in the collection exactly once, which prevents zero divisions).

![](http://www.jamesbowman.me/post/semanticanalysis/tfidf-formula.jpeg)

* n is the total number of documents in the document set.

* df(t) is the number of documents in the document set that contain term.

Let us identify the individual values for the words — potential and wings.

Number of Documents = 8

Number of documents in the corpus that contain the word potential = 1

Number of documents in the corpus that contain the word wings = 2

### Applying the formula for Inverse Document Frequency, we get

### idf(potential) = ln((1+8)/(1+1)) + 1 = ln(9/2) + 1 = 2.50477

### idf(wings) = ln((1+8)/(1+2)) + 1 = ln(9/3) + 1 = 2.0908612

In [12]:
count_vect.idf_

AttributeError: ignored

In [9]:
# explore idf
# idf_ attribute can be used to extract IDF values
# transpose the 1D IDF array to convert to a dataframe to make it easy to visualise
df_idf = idf2df(vectorizer.idf_[:,np.newaxis].T ,terms)
display(HTML(df_idf.to_html()))

NameError: ignored